In [1]:
import openai
import json
import os
import time
import csv
from pathlib import Path

#OpenAI API KEy
openai.api_key = "sk-proj-"


In [2]:
def load_prompt(prompt_path):
    with open(prompt_path, 'r', encoding='utf-8') as f:
        return f.read()

def generate_sentences(prompt_text):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user", "content": prompt_text}
        ],
        temperature=0.7
    )
    content = response["choices"][0]["message"]["content"]
    try:
        return json.loads(content)
    except json.JSONDecodeError:
        print("JSON parse hatası:", content[:20])
        return []


In [3]:
def generate_schema_sentences(prompt_path, output_json_path, output_csv_path, total_needed=30, batch_size=10):
    base_prompt = load_prompt(prompt_path)
    collected = []

    while len(collected) < total_needed:
        needed = min(batch_size, total_needed - len(collected))
        current_prompt = base_prompt.replace("Generate 30", f"Generate {needed}")
        new_sentences = generate_sentences(current_prompt)
        collected.extend([s for s in new_sentences if isinstance(s, str)])
        collected = list(dict.fromkeys(collected))  # Duplicate'leri temizle
        print(f"{prompt_path.name}: {len(collected)}/{total_needed} cümle toplandı.")
        time.sleep(1)

    # JSON olarak kaydet
    with open(output_json_path, "w", encoding="utf-8") as f:
        json.dump(collected[:total_needed], f, ensure_ascii=False, indent=2)

    # CSV olarak da kaydet
    with open(output_csv_path, "w", encoding="utf-8", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["Sentence"])
        for s in collected[:total_needed]:
            writer.writerow([s])

    print(f"Tamamlandı: {prompt_path.stem} → {total_needed} cümle kaydedildi.")

In [4]:
from pathlib import Path

prompt_folder = Path("prompts")
output_folder = Path("test")
output_folder.mkdir(exist_ok=True)


for prompt_file in prompt_folder.glob("*.txt"):  
    name = prompt_file.stem  
    json_path = output_folder / f"{name}.json" 
    csv_path = output_folder / f"{name}.csv"  
    generate_schema_sentences(prompt_file, json_path, csv_path)  


AB.txt: 10/30 cümle toplandı.
JSON parse hatası: [
"I always feel lik
AB.txt: 10/30 cümle toplandı.
AB.txt: 20/30 cümle toplandı.
AB.txt: 30/30 cümle toplandı.
Tamamlandı: AB → 30 cümle kaydedildi.
AS.txt: 10/30 cümle toplandı.
JSON parse hatası: [
"I constantly comp
AS.txt: 10/30 cümle toplandı.
AS.txt: 20/30 cümle toplandı.
AS.txt: 30/30 cümle toplandı.
Tamamlandı: AS → 30 cümle kaydedildi.
DI.txt: 10/30 cümle toplandı.
DI.txt: 20/30 cümle toplandı.
DI.txt: 30/30 cümle toplandı.
Tamamlandı: DI → 30 cümle kaydedildi.
DS.txt: 10/30 cümle toplandı.
DS.txt: 20/30 cümle toplandı.
JSON parse hatası: [
"I constantly feel
DS.txt: 20/30 cümle toplandı.
JSON parse hatası: [
"I constantly feel
DS.txt: 20/30 cümle toplandı.
JSON parse hatası: [
"I constantly feel
DS.txt: 20/30 cümle toplandı.
JSON parse hatası: [
"I constantly feel
DS.txt: 20/30 cümle toplandı.
JSON parse hatası: [
"I constantly feel
DS.txt: 20/30 cümle toplandı.
JSON parse hatası: [
"I constantly feel
DS.txt: 20/30 cümle toplan

In [5]:
import pandas as pd

output_dir = Path("test")
merged_rows = []

for csv_file in output_dir.glob("*.csv"):
    schema_name = csv_file.stem.upper().replace("_", " ")
    df = pd.read_csv(csv_file)
    df.insert(0, "Stage", schema_name)  # İlk sütuna Stage ekle
    merged_rows.append(df)

merged_df = pd.concat(merged_rows, ignore_index=True)
merged_df.to_csv("test_data.csv", index=False, encoding="utf-8")

print(f"Birleştirme tamamlandı: {len(merged_df)} cümle `test_data.csv` dosyasına kaydedildi.")

Birleştirme tamamlandı: 540 cümle `test_data.csv` dosyasına kaydedildi.
